<a href="https://colab.research.google.com/github/adwardkim9/comet-hunting/blob/main/Tensor_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import torch
import torchvision
from astropy.io import fits

## path_csv

In [ ]:
#예제 파일 읽기
train_files = natsort.natsorted(glob.glob('/content/drive/MyDrive/Comet Hunting/Train/cmt*/*.fts'))

In [ ]:
len(train_files)

24195

In [ ]:
split_lists = [string.split(', ') for string in train_files]

In [ ]:
import csv

# File path to save the CSV
file_path = '/content/drive/MyDrive/Comet Hunting/train_file_path.csv'

# Writing data to CSV file
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    for sublist in split_lists:
        writer.writerow(sublist)

# with open(file_path, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     for path in file_path:
#         writer.writerow([path])

print("Data saved to", file_path)

Data saved to /content/drive/MyDrive/Comet Hunting/train_file_path.csv


In [ ]:
import natsort
import glob
import warnings
warnings.filterwarnings("ignore")

## 1

In [ ]:
/content/drive/MyDrive/Comet Hunting/Test/10146987/22546251.fts

In [ ]:
# Function to load FITS images and normalize them
def load_and_normalize_fits_images(folder_path):
    image_tensors = []
    folder_path = pathlib.Path(folder_path)
    for subfolder in folder_path.iterdir():
        if subfolder.is_dir() and subfolder.name.startswith("cmt"):
            subfolder_num = int(subfolder.name[3:])  # Extract the numeric part from the folder name
            if 201 <= subfolder_num <= 300:  # Check if the folder number is within the range
                subfolder_images = []
                for fits_file in subfolder.iterdir():
                    if fits_file.suffix.lower() == '.fts':
                        # Load FITS image
                        image_data, header = fits.getdata(str(fits_file), header=True)
                        # Normalize the image data (you can customize the normalization)
                        image_data = image_data / header['EXPTIME']
                        normalized_image = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
                        # Convert to PIL image
                        image = Image.fromarray(normalized_image)
                        # Convert to tensor
                        image_tensor = torch.tensor(np.array(image), dtype=torch.float32)
                        subfolder_images.append((fits_file.stem, image_tensor))  # Store file name with tensor
                image_tensors.append((subfolder.name, subfolder_images))
    return image_tensors

# Function to save image tensors
def save_image_tensors(image_tensors, save_folder):
    for folder_name, subfolder_images in image_tensors:
        subfolder_save_folder = f"{save_folder}/{folder_name}"
        pathlib.Path(subfolder_save_folder).mkdir(parents=True, exist_ok=True)  # Create folder if not exists
        for file_name, image_tensor in subfolder_images:
            save_path = f"{subfolder_save_folder}/{file_name}.pt"
            torch.save(image_tensor, save_path)

In [ ]:
# Path to the train folder
train_folder = "/content/drive/MyDrive/Comet Hunting/Train/"

# Load and normalize FITS images from train folder
image_tensors = load_and_normalize_fits_images(train_folder)

# Path to save the image tensors
save_folder = "/content/drive/MyDrive/Comet Hunting/Train_image_tensors/"

# Save image tensors
save_image_tensors(image_tensors, save_folder)

print("Train Image tensors saved successfully.")

Train Image tensors saved successfully.


In [ ]:
# Function to load FITS images and normalize them
def load_and_normalize_fits_images_test(folder_path):
    image_tensors = []
    folder_path = pathlib.Path(folder_path)
    for subfolder in folder_path.iterdir():
        if subfolder.is_dir():
            subfolder_images = []
            for fits_file in subfolder.iterdir():
                if fits_file.suffix.lower() == '.fts':
                    # Load FITS image
                    image_data, header = fits.getdata(str(fits_file), header=True)
                    # Normalize the image data (you can customize the normalization)
                    image_data = image_data / header['EXPTIME']
                    normalized_image = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
                    # Convert to PIL image
                    image = Image.fromarray(normalized_image)
                    # Convert to tensor
                    image_tensor = torch.tensor(np.array(image), dtype=torch.float32)
                    subfolder_images.append((fits_file.stem, image_tensor))  # Store file name with tensor
            image_tensors.append((subfolder.name, subfolder_images))
    return image_tensors

In [ ]:
# Path to the test folder
test_folder = "/content/drive/MyDrive/Comet Hunting/Test/"

# Load and normalize FITS images from train-sample folder
image_tensors = load_and_normalize_fits_images_test(test_folder)

# Path to save the image tensors
save_folder = "/content/drive/MyDrive/Comet Hunting/Test_image_tensors/"

# Save image tensors
save_image_tensors(image_tensors, save_folder)

print("Test Image tensors saved successfully.")

KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, data_paths, label_paths):
        self.data_paths = data_paths
        self.label_paths = label_paths
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        # Load tensor data (x)
        x = torch.load(self.data_paths[idx])  # Assuming data_paths contain paths to tensor files

        # Load label from CSV file (y)
        label_df = pd.read_csv(self.label_paths[idx])
        y = label_df['label'].iloc[0]  # Assuming 'label' is the column name containing the label value

        return x, y


In [ ]:
#데이터셋 선언
dataset = CustomDataset(data=files, label=label)

## tensor visualization

In [ ]:
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms, models
from matplotlib import pyplot as plt

In [ ]:

# Load the tensor file
tensor_file_path = "/content/drive/MyDrive/Comet Hunting/sample_image_tensors/cmt0001/22539952.pt"
image_tensor = torch.load(tensor_file_path)

# Visualize the tensor as an image
plt.imshow(image_tensor.numpy(), cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()